# Evaluate the performance of the different models by using a sliding window approach

Steps:
1. Load the ground truth signals and harmonize the data
2. Evaluate the performance of the different models by using a sliding window approach

## Step 1: Load the ground truth signals and harmonize the data

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

In [ ]:
unsupervised_file = utils.join_paths(signals_dir, 'unsupervised_predictions.csv')
unsupervised_predictions = pd.read_csv(unsupervised_file)
unsupervised_predictions['signal'] = unsupervised_predictions['signal'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
unsupervised_predictions = unsupervised_predictions[unsupervised_predictions['roi'] == 'chest']

# Rename column method to model
unsupervised_predictions.rename(columns={'method': 'model'}, inplace=True)

# Only keep the columns that are needed
unsupervised_predictions = unsupervised_predictions[['subject', 'setting', 'model', 'signal']]

unsupervised_predictions.head()

In [ ]:
fine_tuned_path = utils.join_paths(signals_dir, 'fine_tuned_predictions.csv')

fine_tuned_prediction = pd.read_csv(fine_tuned_path)
fine_tuned_prediction['signal'] = fine_tuned_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
fine_tuned_prediction = fine_tuned_prediction[['subject', 'setting', 'model', 'signal']]

fine_tuned_prediction.head()

## Step 2: Evaluate the performance of the different models by using a sliding window approach

In [ ]:
predictions = pd.concat([
    raft_predictions,
    pretrained_predictions,
    unsupervised_predictions,
    fine_tuned_prediction,
])
len(predictions)

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import itertools
import scipy.stats as stats
import respiration.analysis as analysis

# rPPG lowpass and highpass frequencies
# lowpass = 0.7
# highpass = 2.5

# Breathing lowpass and highpass frequencies
lowpass = 0.08
highpass = 0.5

evaluation = []

for model in predictions['model'].unique():
    model_predictions = predictions[predictions['model'] == model]

    gt_psds = []
    pred_psds = []

    subjects = model_predictions['subject'].unique()
    settings = model_predictions['setting'].unique()

    for (subject, setting) in itertools.product(subjects, settings):
        # Get the predicted signal
        pred_signal = model_predictions[
            (model_predictions['subject'] == subject) &
            (model_predictions['setting'] == setting)]['signal'].values[0]

        # Get the ground truth signal
        gt_signal = dataset.get_vital_sign(subject, setting, utils.VitalSigns.thorax_abdomen)

        # Remove the first entry of the ground truth signal, because the prediction is run on diff of the signal
        gt_signal = gt_signal[1:]

        prediction_windows = analysis.sliding_window_psd(pred_signal, 30, lowpass, highpass)
        gt_psds.extend(prediction_windows)

        gt_windows = analysis.sliding_window_psd(gt_signal, 30, lowpass, highpass)
        pred_psds.extend(gt_windows)

    gt_psds = np.array(gt_psds)
    pred_psds = np.array(pred_psds)

    mae = np.mean(np.abs(gt_psds - pred_psds))
    rmse = np.sqrt(np.mean((gt_psds - pred_psds) ** 2))
    corr, p = stats.pearsonr(gt_psds.flatten(), pred_psds.flatten())

    evaluation.append({
        'model': model,
        'mae': mae,
        'rmse': rmse,
        'corr': corr,
        'p': p
    })

In [ ]:
evaluation_df = pd.DataFrame(evaluation)
evaluation_df